In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline
import glob
import os
from collections import Counter
import pandas as pd
import time
import cv2

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential

In [ ]:
def load_data(path):
    x_train=[]
    y_train=[]
    x_test=[]
    y_test=[]
    split=0.1

    folders=os.listdir(path)
    for folder in folders:
        i=0
        images_in_folder=os.listdir(path+folder)
        split_size=len(images_in_folder)*split
        for image in images_in_folder:
            i+=1
            #img=Image.open(path+folder+'/'+image)#.convert('LA')
            img=cv2.imread(path+folder+'/'+image,cv2.IMREAD_GRAYSCALE)
            #ret,thresh_img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
            thresh_img=cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                            cv2.THRESH_BINARY,11,2)
            image_array=np.array(thresh_img)
            #gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)
            image_array=cv2.resize(image_array,(100,100))
            if i>=split_size:
                x_train.append(image_array)
                y_train.append(folder)
            else:
                x_test.append(image_array)
                y_test.append(folder)

    return (np.array(x_train),np.array(y_train)),(np.array(x_test),np.array(y_test))

In [ ]:
img=cv2.imread('C:/Users/hp/machine_learning/mouse_clicking/training_data/fist/pic_005.jpg')
image_array=np.array(img)
gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

In [ ]:
img.shape

In [ ]:
batch_size = 50
num_classes = 2
epochs = 5
img_x, img_y = 100, 100
path="C:/Users/hp/machine_learning/mouse_clicking/train1/"

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data(path)

In [ ]:
x_train[0].shape

In [ ]:
y_train[y_train=="palm"]=0
y_train[y_train=="fist"]=1
y_test[y_test=="palm"]=0
y_test[y_test=="fist"]=1

In [ ]:
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

In [ ]:
history = AccuracyHistory()

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
plt.plot(range(1, epochs+1), history.acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
model_yaml = model.to_yaml()
with open("fist_palm_cnn_model_binary4.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("fist_palm_cnn_model_binary4.h5")
print("Saved model to disk")